In [ ]:
# Starter imports
import sqlite3

# TODO: add sample schema, queries, and index/EXPLAIN demos

## SQLite Mini Demo
- Create a table, insert rows, index a column, and compare query plans.

In [ ]:
import sqlite3

conn = sqlite3.connect(":memory:")
cur = conn.cursor()
cur.executescript(
    """
    CREATE TABLE user(id INTEGER PRIMARY KEY, name TEXT, city TEXT);
    INSERT INTO user(name, city) VALUES ('ada', 'london'), ('grace', 'baltimore');
    CREATE INDEX idx_user_city ON user(city);
    """
)

cur.execute("SELECT * FROM user WHERE city=?", ("london",))
rows = cur.fetchall()

plan = list(cur.execute("EXPLAIN QUERY PLAN SELECT * FROM user WHERE city='london'"))
rows, plan[:2]

Notes:
- Use indexes on selective columns; check `EXPLAIN QUERY PLAN` to verify index use.
- Keep transactions short; prefer parameterized queries to avoid injection.

## Transactions and Constraints
- Use transactions to keep multi-step writes atomic; roll back on failure.
- Constraints (NOT NULL, UNIQUE, FK) protect integrity even when app code errs.

In [ ]:
import sqlite3

conn = sqlite3.connect(":memory:")
cur = conn.cursor()
cur.executescript("""
CREATE TABLE account(id INTEGER PRIMARY KEY, email TEXT UNIQUE NOT NULL);
INSERT INTO account(email) VALUES ('a@example.com');
""")

try:
    cur.execute("INSERT INTO account(email) VALUES (?)", ("a@example.com",))
    conn.commit()
except sqlite3.IntegrityError as exc:
    conn.rollback()
    dup_error = str(exc)

cur.execute("SELECT id, email FROM account")
rows = cur.fetchall()
dup_error, rows

## CRUD Essentials
- Create: define schemas/tables with types, constraints, defaults. In SQL, `CREATE TABLE` and `ALTER TABLE` evolve structure safely.
- Read: select only needed columns/rows; use indexes for predicates/sorts; measure with `EXPLAIN`.
- Update: scoped updates with predicates; beware write amplification and locks; keep transactions short.
- Delete: prefer soft-delete or archival tables for audit; `DELETE` vs `TRUNCATE` vs `DROP` have very different effects.

## Relational vs Non-Relational
- Relational: strong schemas, joins, ACID guarantees; great for consistency, reporting, and ad-hoc queries.
- Document/Key-Value: flexible schemas, denormalized aggregates; simpler horizontal scaling, but you handle consistency at app level.
- Columnar: optimized for analytics scans and aggregations (OLAP).
- Graph: optimized for traversals and relationship queries.

Guidance: start relational by default; move specific workloads (analytics → columnar; high fan-out aggregates → doc/kv; deep traversals → graph). Keep invariants close to where data is written (constraints, triggers, or validated write paths).

In [ ]:
# CRUD walkthrough in SQLite
import sqlite3
conn = sqlite3.connect(":memory:")
cur = conn.cursor()
cur.execute("CREATE TABLE item(id INTEGER PRIMARY KEY, name TEXT, qty INTEGER)")

# Create/Insert
cur.executemany("INSERT INTO item(name, qty) VALUES (?, ?)", [("apple", 3), ("banana", 5)])

# Read
cur.execute("SELECT name, qty FROM item ORDER BY name")
rows_before = cur.fetchall()

# Update
cur.execute("UPDATE item SET qty = qty + 1 WHERE name = ?", ("apple",))

# Delete
cur.execute("DELETE FROM item WHERE name = ?", ("banana",))
cur.execute("SELECT name, qty FROM item ORDER BY name")
rows_after = cur.fetchall()
rows_before, rows_after